In [1]:
import numpy as np
import pandas as pd
import random
import math

Config

In [2]:
def create_data_model(path):
    #Store the data model of the problem
    data = {}
    with open (path, "r") as f:
        lines = f.readlines()
        line = list(map(int,lines[0].strip().split()))
        N = line[0]
        M = line[1]
        SIGMA = N + M
        K = line[2]
        p = np.zeros(2*SIGMA +1, dtype='int')
        q = np.zeros(2*SIGMA +1, dtype='int')
        Q = np.zeros(K+1, dtype='int')
        d = np.zeros((2*SIGMA +1, 2*SIGMA+1), dtype='int')

        line = list(map(int,lines[1].strip().split()))
        for m in range(1,2*SIGMA +1):
            if (m<=N):
                p[m] = 1
            elif (m> SIGMA and m <= SIGMA + N):
                p[m] = -1
            if (m > N and m <= SIGMA):
                q[m] = line[m-N-1]
            elif (m>SIGMA+N and m <=2*SIGMA):
                q[m] = line[m-SIGMA-N-1]*(-1)
            else :
                q[m] = 0
                

        line = list(map(int,lines[2].strip().split()))
        for k in range(1,K+1):
            Q[k] = line[k-1]

        for i in range (0, 2*SIGMA+1):
            line = list(map(int,lines[i+3].strip().split()))
            for j in range(0, 2*SIGMA+1):
                d[i][j] = line[j]

        data['N'] = N
        data['M'] = M
        data['q'] = q.tolist()
        data['p'] = p.tolist()
        data['Q'] = Q.tolist()
        data['d'] = d.tolist()
        data['K'] = K
        data['root'] = 0
        data['SIGMA'] = SIGMA
    return data


In [5]:
data = create_data_model('../../res/data.text')
data

{'N': 2,
 'M': 3,
 'q': [0, 0, 0, 3, 8, 12, 0, 0, -3, -8, -12],
 'p': [0, 1, 1, 0, 0, 0, -1, -1, 0, 0, 0],
 'Q': [0, 15, 20, 13],
 'd': [[0, 3, 4, 5, 6, 3, 4, 5, 6, 4, 3],
  [3, 0, 4, 5, 6, 3, 4, 5, 6, 12, 2],
  [1, 3, 0, 5, 6, 3, 34, 5, 6, 4, 1],
  [8, 3, 4, 0, 6, 3, 4, 5, 6, 11, 21],
  [9, 3, 4, 5, 0, 3, 4, 5, 6, 32, 13],
  [2, 3, 44, 5, 33, 0, 4, 5, 6, 43, 4],
  [3, 3, 4, 5, 6, 34, 0, 5, 6, 23, 1],
  [10, 4, 5, 6, 3, 4, 0, 6, 55, 33, 32],
  [14, 3, 4, 5, 6, 3, 4, 5, 0, 5, 31],
  [4, 5, 6, 32, 0, 4, 5, 6, 2, 4, 2],
  [4, 5, 6, 32, 0, 4, 5, 6, 2, 4, 2]],
 'K': 3,
 'root': 0,
 'SIGMA': 5}

Global variable

In [72]:
class Individual:
    def __init__(self, data):
        self.SIGMA = data['SIGMA']
        self.K = data['K']
        self.serves = []
        

In [73]:
def weight(path):
    weight = 0
    for i in range(1, N):
        weight = weight + D[path[i-1]][path[i]]

    weight = weight+ D[path[N-1]][path[0]]
    return weight

In [ ]:
def checkSolution(s:Sample):
    check = True
    for i in range(N):
        if i not in s.path:
            check = False
            break
    return check

: 

In [75]:
population = []
def init_population():
    global M, N, population
    population = []
    for i in range(M):
        path = random.sample(range(N),N)
        w = weight(path)
        sample = Sample()
        sample.path = path
        sample.weight = w
        population.append(sample)

Operator

In [76]:
def hybridize(s1:Sample, s2: Sample):
    s3 = Sample()
    p = random.randint(0, N-1)

    for i in range (p, N):
        s3.path[i-p] = s2.path[i]

    k = N-p
    for a in s1.path:
        if a not in s3.path:
            s3.path[k] = a
            k = k+1
            if (k >= N ) :break
    s3.weight = weight(s3.path)
    return s3


In [77]:
def mutate(s:Sample):
    s_child = Sample()
    s_child.path = s.path.copy()
    mutateCoefficient = int(N*0.1)
    count = random.randint(0, mutateCoefficient)
    for i in range(count):
        p1 = random.randint(0,N-1)
        p2 = random.randint(0,N-1)

        temp = s_child.path[p1]
        s_child.path[p1] = s_child.path[p2]
        s_child.path[p2] = temp

    s_child.weight = weight(s_child.path)
    return s_child


In [78]:
def sortPopulation():
    global population
    

GA algorithm

In [88]:
def GA(fitness_threshold, M, r, m):
    global population

    solution = Sample()
    solution.weight = 9999999

    init_population()
    population.sort(key= lambda x: x.weight)

    num_standard_sample = int((1-r)*M)
    num_hybrid_sample = M - num_standard_sample

    while (fitness_threshold > 0):
        fitness_threshold = fitness_threshold -1
        #Hybridize
        new_population = []
        new_population = population[: num_standard_sample]

        hybridize_pop = random.sample(population, num_hybrid_sample)
        random.shuffle(hybridize_pop)

        i = 0
        while i < int(num_hybrid_sample):
            s1 = hybridize(hybridize_pop[i], hybridize_pop[i+1])
            s2 = hybridize(hybridize_pop[i+1], hybridize_pop[i])
            new_population.append(s1)
            new_population.append(s2)
            i = i + 2

        for sample in new_population:
            a1 = mutate(sample)
            sample = a1

        population = new_population
        population.sort(key= lambda x: x.weight)

        if (population[0].weight < solution.weight):
            solution.path = population[0].path
            solution.weight = population[0].weight

        
    return solution


In [89]:
solution = GA(1000, M, 0.5, 0.2)

In [90]:
print(solution.path)
print(solution.weight)


[26  5 25 30 27  7 47 22  6 42 23 13 24 17  3 12 40 18 39 41 16 46 45 10
 31 21  0  1 19  2 35 34 28 20 49 33  8 15 37  4 48 38 29  9 32 44 14 43
 36 11 50]
503.00533817651944


In [91]:
solution = GA(5000, M, 0.5, 0.2)
print(solution.path)
print(solution.weight)

[28 15 10 31 50 45 11 46  3 17 16 36 14 32 44 43 41 18 39 40 12 24 13 23
 42  6 22  5 47 26  0 21 27  7 25 30  2 35 34 19  1 37  4 48  9 38 29 33
  8 49 20]
466.7547525135038


In [92]:
solution = GA(10000, M, 0.5, 0.2)
print(solution.path)
print(solution.weight)

[30 25  7 47  5 22  6 42 23 13 24 17 46 11 45 50 26  1 28 20 15 49 33 29
  8 48  9 38 32 44 14 43 41 18 39 40 12  3 16 36  4 37 10 31  0 21 19 34
 35  2 27]
446.6227640876554


In [93]:
solution = GA(100000, M, 0.5, 0.2)
print(solution.path)
print(solution.weight)

[37  4 11 46 17  3 41 18 39 40 12 24 13 23 42  6 22 47  5 26 45 50 16 36
 43 14 44 32 38  9 48  8 49 15 19 34 35  2 27 30 25  7 21  0 31 10  1 28
 20 33 29]
458.47354261492757
